# Final Exam Challenge for Microsoft Professional Program - Principles of Machine Learning with Python

## Part 2 - Classification

Use the Adventure Works Cycles customer data you worked with in challenge 1 to create a classification model that predicts whether or not a customer will purchase a bike. The model should predict bike purchasing for new customers for whom no information about average monthly spend or previous bike purchases is available.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
import seaborn as sns
import numpy as np
import numpy.random as nr
import math
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/matplotlib/font_manager.py:281: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [19]:
avgmonthlyspend = pd.read_csv('/home/nbuser/library/FinalExam/AW_AveMonthSpend.csv')
bikebuyer = pd.read_csv('/home/nbuser/library/FinalExam/AW_BikeBuyer.csv')
customers = pd.read_csv('/home/nbuser/library/FinalExam/AdvWorksCusts.csv')
validation_customers = pd.read_csv('/home/nbuser/library/FinalExam/AW_test.csv')
combined_df = customers.join(avgmonthlyspend.set_index('CustomerID'),on='CustomerID')
combined_df2 = combined_df.join(bikebuyer.set_index('CustomerID'),on='CustomerID')

In [20]:
reduced_df = combined_df2[['BirthDate','Education','Occupation','Gender','MaritalStatus','HomeOwnerFlag','NumberCarsOwned','NumberChildrenAtHome','TotalChildren','YearlyIncome','AveMonthSpend','BikeBuyer']]
reduced_df.head()

BirthDate   Education    Occupation Gender MaritalStatus  HomeOwnerFlag  \
0  1966-04-08  Bachelors   Professional      M             M              1   
1  1965-05-14  Bachelors   Professional      M             S              0   
2  1965-08-12  Bachelors   Professional      M             M              1   
3  1968-02-15  Bachelors   Professional      F             S              0   
4  1968-08-08  Bachelors   Professional      F             S              1   

   NumberCarsOwned  NumberChildrenAtHome  TotalChildren  YearlyIncome  \
0                0                     0              2        137947   
1                1                     3              3        101141   
2                1                     3              3         91945   
3                1                     0              0         86688   
4                4                     5              5         92771   

   AveMonthSpend  BikeBuyer  
0             89          0  
1            117          1  
2            123          0  
3             50          0  
4             95          1

In [21]:
combined_validation_cust = validation_customers.join(avgmonthlyspend.set_index('CustomerID'),on='CustomerID').join(bikebuyer.set_index('CustomerID'),on='CustomerID')

In [22]:
reduced_validation = combined_validation_cust[['BirthDate','Education','Occupation','Gender','MaritalStatus','HomeOwnerFlag','NumberCarsOwned','NumberChildrenAtHome','TotalChildren','YearlyIncome','AveMonthSpend','BikeBuyer']]
reduced_validation.head()

BirthDate        Education      Occupation Gender MaritalStatus  \
0   1/5/1945       Bachelors       Management      F             S   
1  10/4/1964       Bachelors   Skilled Manual      M             M   
2  1/12/1934  Graduate Degree      Management      F             M   
3  9/22/1958  Graduate Degree  Skilled Manual      M             M   
4  3/19/1965      High School          Manual      F             M   

   HomeOwnerFlag  NumberCarsOwned  NumberChildrenAtHome  TotalChildren  \
0              0                2                     0              5   
1              1                2                     2              4   
2              1                2                     0              4   
3              1                0                     0              4   
4              1                1                     2              2   

   YearlyIncome  AveMonthSpend  BikeBuyer  
0         86931            NaN        NaN  
1        100125            NaN        NaN  
2        103985            NaN        NaN  
3        127161            NaN        NaN  
4         21876            NaN        NaN

In [23]:
def count_missing_values(df, missing_value_indicator):
    """For each column in a dataframe df print the number of rows
    where the value matches the missing value indicator """
    for col in df.columns:
        if df[col].dtype == object:
            count = 0
            count = [count + 1 for x in df[col] if x == missing_value_indicator]
            print(col + ' ' + str(sum(count)))

In [26]:
from datetime import datetime

def age(birth_date,dateformat='%Y-%m-%d'):
    dob = datetime.strptime(birth_date, dateformat)
    ref_date = datetime.strptime('1998-01-01', dateformat)
    years = ref_date.year - dob.year
    if ref_date.month < dob.month or (ref_date.month == dob.month and ref_date.day < dob.day):
        years -= 1
    return years

def age2(birth_date,dateformat='%m/%d/%Y'):
    dob = datetime.strptime(birth_date, dateformat)
    ref_date = datetime.strptime('01-01-1998', dateformat)
    years = ref_date.year - dob.year
    if ref_date.month < dob.month or (ref_date.month == dob.month and ref_date.day < dob.day):
        years -= 1
    return years

In [27]:
reduced_df['Age'] = reduced_df['BirthDate'].apply(age)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [28]:
reduced_validation['Age'] = reduced_validation['BirthDate'].apply(age2)
reduced_validation.drop(['BirthDate'], axis=1, inplace=True)

ValueError: time data '1998-01-01' does not match format '%m/%d/%Y'

In [13]:
reduced_df.drop(['BirthDate'], axis=1, inplace=True)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [14]:
reduced_df.head()

Education    Occupation Gender MaritalStatus  HomeOwnerFlag  \
0  Bachelors   Professional      M             M              1   
1  Bachelors   Professional      M             S              0   
2  Bachelors   Professional      M             M              1   
3  Bachelors   Professional      F             S              0   
4  Bachelors   Professional      F             S              1   

   NumberCarsOwned  NumberChildrenAtHome  TotalChildren  YearlyIncome  \
0                0                     0              2        137947   
1                1                     3              3        101141   
2                1                     3              3         91945   
3                1                     0              0         86688   
4                4                     5              5         92771   

   AveMonthSpend  BikeBuyer  Age  
0             89          0   31  
1            117          1   32  
2            123          0   32  
3             50          0   29  
4             95          1   29

In [9]:
count_missing_values(reduced_df,"")

BirthDate 0
Education 0
Occupation 0
Gender 0
MaritalStatus 0


In [16]:
def encode_string(cat_feature):
    """
    categorical_columns = ['col1_name', 'col2_name', 'col3_name']
    for col in categorical_columns:
        temp = encode_string(df[col])
        Features = np.concatenate([Features, temp], axis = 1)
    """
    ## First encode the strings to numeric categories
    enc = preprocessing.LabelEncoder()
    enc.fit(cat_feature)
    enc_cat_feature = enc.transform(cat_feature)
    ## Now, apply one hot encoding
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_cat_feature.reshape(-1,1))
    return encoded.transform(enc_cat_feature.reshape(-1,1)).toarray()

In [22]:
labels = np.array(reduced_df['BikeBuyer'])

In [24]:
Features = encode_string(reduced_df['Education'])
categorical_columns = ['Occupation','Gender','MaritalStatus']
for col in categorical_columns:
    temp = encode_string(reduced_df[col])
    Features = np.concatenate([Features, temp], axis=1)

In [25]:
Features = np.concatenate([Features, np.array(reduced_df[['HomeOwnerFlag','NumberCarsOwned','NumberChildrenAtHome','TotalChildren','YearlyIncome','AveMonthSpend','Age']])], axis = 1)

In [29]:
print(Features.shape)

(17209, 21)


In [30]:
print(Features[:2, :]) 

[[1.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00
  1.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00 0.00000e+00 2.00000e+00
  1.37947e+05 8.90000e+01 3.10000e+01]
 [1.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00
  0.00000e+00 1.00000e+00 0.00000e+00 1.00000e+00 3.00000e+00 3.00000e+00
  1.01141e+05 1.17000e+02 3.20000e+01]]


In [33]:
nr.seed(9988)
indx = range(Features.shape[0])
indx = ms.train_test_split(indx, test_size = 300)
X_train = Features[indx[0],:]
y_train = np.ravel(labels[indx[0]])
X_test = Features[indx[1],:]
y_test = np.ravel(labels[indx[1]])

Q: What is the purpose of the slicing like X_train[:,19:] >> only want to scale certain features


In [39]:
#X_train[:5,:]
X_train[:2,]

array([[0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 1.00000e+00, 1.00000e+00, 0.00000e+00, 1.00000e+00,
        2.00000e+00, 0.00000e+00, 2.00000e+00, 8.11730e+04, 6.80000e+01,
        6.20000e+01],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 1.00000e+00, 0.00000e+00, 1.00000e+00, 1.00000e+00,
        2.00000e+00, 0.00000e+00, 4.00000e+00, 1.09129e+05, 6.80000e+01,
        6.20000e+01]])

In [40]:
X_train[:2,]
scaler = preprocessing.StandardScaler().fit(X_train[:,18:19])
X_train[:,18:19] = scaler.transform(X_train[:,18:19])
X_test[:,18:19] = scaler.transform(X_test[:,18:19])
X_train[:2,]

array([[ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  1.        ,  0.        ,  1.        ,
         2.        ,  0.        ,  2.        ,  0.07794072, 68.        ,
        62.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  1.        ,
         2.        ,  0.        ,  4.        ,  0.78384066, 68.        ,
        62.        ]])

In [41]:
logistic_mod = linear_model.LogisticRegression() 
logistic_mod.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [42]:
probabilities = logistic_mod.predict_proba(X_test)
print(probabilities[:15,:])

[[0.37645264 0.62354736]
 [0.94744723 0.05255277]
 [0.84360738 0.15639262]
 [0.96624181 0.03375819]
 [0.07292901 0.92707099]
 [0.84719301 0.15280699]
 [0.94944892 0.05055108]
 [0.06093302 0.93906698]
 [0.19203764 0.80796236]
 [0.11419903 0.88580097]
 [0.87455408 0.12544592]
 [0.38433205 0.61566795]
 [0.96450435 0.03549565]
 [0.92731391 0.07268609]
 [0.68983657 0.31016343]]


In [44]:
def score_model(probs, threshold):
    return np.array([1 if x > threshold else 0 for x in probs[:,1]])
scores = score_model(probabilities, 0.5)
print(np.array(scores[:]))
print(y_test[:])

[1 0 0 0 1 0 0 1 1 1 0 1 0 0 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 1 0 1 0
 1 0 1 0 1 1 1 0 0 0 1 0 0 0 1 0 0 1 0 1 0 0 0 1 0 1 0 1 1 1 0 1 0 0 1 1 0
 0 1 1 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 0 1 1 1 1 1 1 0 0
 0 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 0 1 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0
 0 0 0 0]
[1 0 0 0 1 0 0 1 1 1 1 0 0 0 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 1 0 1 0
 0 0 1 0 1 0 1 0 0 0 1 0 0 0 1 0 1 1 0 0 1 0 0 1 0 1 1 1 1 0 1 1 0 1 1 0 0
 0 1 1 0 1 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 1 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 1 0 0 0 1 1 0 0 1 1 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 1
 0 0 0 0 1 0 0 

In [45]:
def print_metrics(labels, scores):
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score positive    Score negative')
    print('Actual positive    %6d' % conf[0,0] + '             %5d' % conf[0,1])
    print('Actual negative    %6d' % conf[1,0] + '             %5d' % conf[1,1])
    print('')
    print('Accuracy  %0.2f' % sklm.accuracy_score(labels, scores))
    print(' ')
    print('           Positive      Negative')
    print('Num case   %6d' % metrics[3][0] + '        %6d' % metrics[3][1])
    print('Precision  %6.2f' % metrics[0][0] + '        %6.2f' % metrics[0][1])
    print('Recall     %6.2f' % metrics[1][0] + '        %6.2f' % metrics[1][1])
    print('F1         %6.2f' % metrics[2][0] + '        %6.2f' % metrics[2][1])


    
print_metrics(y_test, scores)  

                 Confusion matrix
                 Score positive    Score negative
Actual positive       183                19
Actual negative        39                59

Accuracy  0.81
 
           Positive      Negative
Num case      202            98
Precision    0.82          0.76
Recall       0.91          0.60
F1           0.86          0.67


In [3]:
validation_df = pd.read_csv('/home/nbuser/library/FinalExam/AW_test.csv')
validation_df.head()

CustomerID Title FirstName MiddleName  LastName Suffix  \
0       18988   NaN  Courtney          A     Baker    NaN   
1       29135   NaN      Adam          C     Allen    NaN   
2       12156   NaN    Bonnie        NaN      Raji    NaN   
3       13749   NaN     Julio          C    Alonso    NaN   
4       27780   NaN   Christy          A  Andersen    NaN   

             AddressLine1 AddressLine2        City StateProvinceName  \
0   8727 Buena Vista Ave.          NaN     Fremont        California   
1        3491 Cook Street          NaN       Haney  British Columbia   
2    359 Pleasant Hill Rd          NaN     Burbank        California   
3        8945 Euclid Ave.          NaN  Burlingame        California   
4  42, boulevard Tremblay          NaN   Dunkerque              Nord   

       ...       BirthDate        Education      Occupation Gender  \
0      ...        1/5/1945       Bachelors       Management      F   
1      ...       10/4/1964       Bachelors   Skilled Manual      M   
2      ...       1/12/1934  Graduate Degree      Management      F   
3      ...       9/22/1958  Graduate Degree  Skilled Manual      M   
4      ...       3/19/1965      High School          Manual      F   

  MaritalStatus HomeOwnerFlag NumberCarsOwned NumberChildrenAtHome  \
0             S             0               2                    0   
1             M             1               2                    2   
2             M             1               2                    0   
3             M             1               0                    0   
4             M             1               1                    2   

   TotalChildren  YearlyIncome  
0              5         86931  
1              4        100125  
2              4        103985  
3              4        127161  
4              2         21876  

[5 rows x 23 columns]